In [ ]:
# get traces and rois from suite2p
%matplotlib widget

In [ ]:
import json
from pathlib import Path
import numpy as np
import flammkuchen as fl
import matplotlib.pyplot as plt 
from fimpylab.core.lightsheet_experiment import LightsheetExperiment

In [ ]:
master_path = Path(r"\\Funes\Shared\experiments\E0044_spontaneous\gad1b\ls\e0044_v00")
master_path = Path(r"\\Funes\shared\experiments\E0040_motions_cardinal\v13_cw_ccw\ls_fixed")
master_path = Path(r"\\Funes\Shared\experiments\E0020_random_coherences\v35_two_options_forward_ol\h2b")
files = list(master_path.glob("*_f*"))
fish_dir = files[2] 
print(fish_dir)

In [ ]:
suite2p_data = fl.load(fish_dir / "data_from_suite2p_cells.h5")
coords = suite2p_data["coords"]
traces = suite2p_data["traces"]
#rois_stack = suite2p_data["rois_stack"]
print(coords)

try:
    with open(next(fish_dir.glob("*metadata.json"))) as i:
        metadata = json.load(i)
    fish_id = metadata['general']['fish_id']
except:
    fish_id = ""
print(fish_dir)
print(fish_id)

In [ ]:
fs = 2
exp = LightsheetExperiment(fish_dir)
beh_log = exp.behavior_log
tail = np.asarray(beh_log['tail_sum'])
t = np.asarray(beh_log["t"]) / fs

In [ ]:
#cell_ind = np.load(f / 'iscell.npy')
#all_traces = np.load(f / 'F.npy')
#traces =all_traces[cell_ind[:,0]==1]

In [ ]:
num_traces, len_rec = np.shape(traces)
print(np.shape(traces))

In [ ]:
fix_traces = np.copy(traces)
# remove_ind = np.where(np.nanstd(traces, 0) == 0)
# print(remove_ind)

remove_ind = []
for i in range(num_traces):
    tmp_trace = fix_traces[i]
    #print(tmp_trace)
    if np.nanstd(tmp_trace) == 0:
        remove_ind.append(i)
print(remove_ind)

#fix_traces = np.delete(fix_traces, (remove_ind), axis=0)

In [ ]:
num_traces, len_rec = np.shape(fix_traces)
print(np.shape(fix_traces))

In [ ]:
norm_traces = np.copy(fix_traces)
sd=np.nanstd(norm_traces, 0)
mean=np.nanmean(norm_traces, 0)
norm_traces=norm_traces-mean 
norm_traces=norm_traces/sd
print(np.shape(norm_traces))

In [ ]:
## sort traces according to z location 
z = coords[:,0]
print(z)

In [ ]:
fig1 = plt.figure(figsize=(10,7))
plt.imshow(fix_traces, aspect="auto", vmin=np.percentile(fix_traces, 1), vmax=np.percentile(fix_traces, 99), cmap="coolwarm")
plt.title(fish_id)
file_name = "traces" + fish_id + ".jpg"
fig1.savefig(str(fish_dir / file_name), dpi=300)



In [ ]:
print(np.percentile(norm_traces, 99))
print(np.percentile(norm_traces, 5))

In [ ]:
fig2, (a0, a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [1, 8]}, figsize=(10,5))
a0.plot(t, tail, linewidth=1)
a0.axis('off')
a1.imshow(norm_traces, aspect="auto", vmin=np.percentile(norm_traces, 1), vmax=np.percentile(norm_traces, 99), cmap="coolwarm")
plt.suptitle(fish_id)
file_name = "norm_traces" + fish_id + ".jpg"
fig2.tight_layout()
fig2.savefig(str(fish_dir / file_name), dpi=300)

In [ ]:
corr_mat = np.corrcoef(norm_traces)


In [ ]:
np.shape(corr_mat)

In [ ]:
fig3 = plt.figure(figsize=(10,7))
plt.imshow(corr_mat, aspect="auto", vmin=-0.3, vmax=0.3, cmap="coolwarm")
plt.title(fish_id)
file_name = "corrmat" + fish_id + "_211031.jpg"
fig3.savefig(str(fish_dir / file_name), dpi=300)